# [Projet 6: Catégorisez automatiquement des questions](https://openclassrooms.com/fr/projects/categorisez-automatiquement-des-questions)
(parcours data: [here](https://openclassrooms.com/paths/63-data-scientist))

## Intelligence

Resources:
- [tutorial on kaggle](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words) (I did this basically on my own, but look at the features with scikit-learn section)

### Imports

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'Documents/GitHub/OCDataSciencePath/Project6/Work/')    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### Data

In [3]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToDataDir = HOME+'Documents/Dropbox/Transit/OCDataScienceData/Project6/'    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))

In [13]:
filename = 'QueryResults_10k_forLearning.npz'

tmp = np.load(pathToDataDir+filename)
x_train,x_test,y_train,y_test = tmp['arr_0'],tmp['arr_1'],tmp['arr_2'],tmp['arr_3']

In [14]:
x_train

array(<788x25 sparse matrix of type '<class 'numpy.float64'>'
	with 3634 stored elements in COOrdinate format>, dtype=object)

In [15]:
x_test

array(<198x25 sparse matrix of type '<class 'numpy.float64'>'
	with 1021 stored elements in COOrdinate format>, dtype=object)

In [16]:
y_train

array(<788x20 sparse matrix of type '<class 'numpy.int64'>'
	with 783 stored elements in Compressed Sparse Row format>, dtype=object)

In [18]:
y_test

array(<198x20 sparse matrix of type '<class 'numpy.int64'>'
	with 176 stored elements in Compressed Sparse Row format>, dtype=object)

### Extra cleanup (?)

### Try things